In [1]:
import pandas as pd

import fnmatch
import glob
import dropbox as db

In [23]:
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import dbx_token
dbx = db.Dropbox(dbx_token)

path = '/Users/matthiashugli/Dropbox/bucket/suisse-keepers/*.csv'

df = pd.DataFrame()
for filename in glob.glob(path):
    file = pd.read_csv(filename, header=0, delimiter=',', low_memory=False)
    df = df.append(file)

ln: config.py: File exists


# Data Preparation

## Data Cleansing

In [27]:
df = df.dropna(how='all')
print(df.describe)

<bound method NDFrame.describe of           Date  Day         Round Venue Result    Squad         Opponent  \
0   2020-09-13  Sun   Matchweek 1  Away  W 1–0  Everton        Tottenham   
1   2020-09-19  Sat   Matchweek 2  Home  W 5–2  Everton        West Brom   
2   2020-09-26  Sat   Matchweek 3  Away  W 2–1  Everton   Crystal Palace   
3   2020-10-03  Sat   Matchweek 4  Home  W 4–2  Everton         Brighton   
4   2020-10-17  Sat   Matchweek 5  Home  D 2–2  Everton        Liverpool   
5   2020-10-25  Sun   Matchweek 6  Away  L 0–2  Everton      Southampton   
6   2020-11-07  Sat   Matchweek 8  Home  L 1–3  Everton   Manchester Utd   
7   2020-11-22  Sun   Matchweek 9  Away  W 3–2  Everton           Fulham   
8   2020-11-28  Sat  Matchweek 10  Home  L 0–1  Everton     Leeds United   
9   2020-12-05  Sat  Matchweek 11  Away  D 1–1  Everton          Burnley   
10  2020-12-12  Sat  Matchweek 12  Home  W 1–0  Everton          Chelsea   
11  2020-12-19  Sat  Matchweek 14  Home  W 2–1  Everto

In [28]:
file.insert(0, 'filename', filename)

In [38]:
x_saves = df.loc[:, ('Saves', 'SoTA', 'GA', 'PSxG', 'Squad')]
x_saves['Saves%'] = x_saves['Saves'] / x_saves['SoTA']
x_saves['xS'] = x_saves['SoTA'] - x_saves['PSxG']
x_saves['xS%'] = x_saves['xS'] / x_saves['SoTA']
print(x_saves.head(5))

   Saves  SoTA   GA  PSxG    Squad    Saves%   xS     xS%
0    5.0   5.0  0.0   0.6  Everton  1.000000  4.4  0.8800
1    2.0   4.0  2.0   0.5  Everton  0.500000  3.5  0.8750
2    0.0   1.0  1.0   0.7  Everton  0.000000  0.3  0.3000
3    1.0   3.0  2.0   1.2  Everton  0.333333  1.8  0.6000
4    6.0   8.0  2.0   2.7  Everton  0.750000  5.3  0.6625


In [40]:
group_xsaves = x_saves.groupby('Squad').agg({'Saves': 'sum', 'xS': 'sum', 'SoTA': 'sum', 'PSxG': 'sum', 'GA': 'sum'})
group_xsaves['xS%'] = group_xsaves.xS / group_xsaves.SoTA
group_xsaves['saves%'] = group_xsaves.Saves / group_xsaves.SoTA
group_xsaves['shot_performance'] = group_xsaves['save%'] - group_xsaves['xS%']
group_xsaves['PSxG/SoTA'] = group_xsaves['PSxG'] / group_xsaves.SoTA
print(group_xsaves)

         Saves    xS   SoTA  PSxG    GA       xS%    Saves%    saves2  \
Squad                                                                   
Everton   90.0  89.3  128.0  38.7  39.0  0.697656  0.703125  0.695312   

         shot_performance  PSxG/SoTA  
Squad                                 
Everton         -0.002344   0.302344  
